In [ ]:
!pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpma9fljwj
  JVM stdout: /tmp/tmpma9fljwj/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpma9fljwj/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_xfopdw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
# Load data into H2O

# df = h2o.import_file(path=path, header =1)
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('/content/Bank_Personal_Loan_Modelling.csv')
labelEncoder = preprocessing.LabelEncoder()
for col in df.columns:
    df[col] = labelEncoder.fit_transform(df[col])

df = h2o.H2OFrame(df)
df = df.asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
df.describe()

Rows:5000
Cols:14




,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,,,,,,,,,,,,,,
mean,,,,,,,,,,,,,,
maxs,,,,,,,,,,,,,,
sigma,,,,,,,,,,,,,,
zeros,,,,,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,2,4,33,83,3,19,0,0,0,1,0,0,0
1,1,22,22,22,34,2,18,0,0,0,1,0,0,0
2,2,16,18,3,367,0,12,0,0,0,0,0,0,0


In [ ]:
train,test = df.split_frame(ratios=[.7])

In [ ]:
y = "Personal Loan"
x_train = train.columns
x_train.remove(y)

aml = H2OAutoML(max_runtime_secs=600, seed = 1)
aml.train(x = x_train, y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200830_172751,0.991306,0.0673615,0.942167,0.0762881,0.133349,0.017782
StackedEnsemble_BestOfFamily_AutoML_20200830_172751,0.991041,0.0714592,0.934955,0.0497576,0.138852,0.0192799
XRT_1_AutoML_20200830_172751,0.986523,0.143781,0.909569,0.081742,0.206101,0.0424776
GBM_grid__1_AutoML_20200830_172751_model_19,0.963943,0.16405,0.852761,0.123975,0.210733,0.0444083
GBM_grid__1_AutoML_20200830_172751_model_6,0.962282,0.160405,0.850781,0.141536,0.202807,0.0411308
GBM_grid__1_AutoML_20200830_172751_model_17,0.961873,0.17001,0.835048,0.144847,0.216337,0.0468015
GBM_3_AutoML_20200830_172751,0.959125,0.18499,0.815538,0.159184,0.217832,0.047451
GBM_grid__1_AutoML_20200830_172751_model_21,0.954122,0.153155,0.800929,0.161169,0.203346,0.0413495
DRF_1_AutoML_20200830_172751,0.953759,0.21098,0.807837,0.171404,0.213203,0.0454553
GBM_grid__1_AutoML_20200830_172751_model_10,0.952953,0.164407,0.844947,0.120433,0.199877,0.0399509


In [ ]:
preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%
